In [1]:
# !pip install --upgrade pip
# !pip install gdown
# import gdown
# %pwd
# %ls
# %cd /content
# %cd drive
# %cd MyDrive
# %mkdir -p Satellite
# %cd Satellite
# !nvidia-smi
# url = 'https://drive.google.com/file/d/1hCM96mRJeYiivFFtQJcTK6w6nYh8GJ-Q'
# gdown.download_folder(url)
# !pip install pytorch_lightning
# !pip update pytorch_lightning
# from google.colab import drive
# drive.mount('/content/drive/')

# import sys
# sys.path.append('drive/MyDrive/LuttingerWard_Prediction/')
# # import data
# # # import drive.MyDrive.LuttingerWard_Prediction.data

# !ls drive/MyDrive/LuttingerWard_Prediction

In [3]:
#%%
import torch
import models
import wrappers
from torch.utils.data import DataLoader
import load_data
import datetime
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.plugins.environments import LightningEnvironment
import json
import os

In [4]:
def train():
    ### JSON File contains full information about entire run (model, data, hyperparameters)
    ### TODO 
    MODEL_NAME = "CONVERGENCE_AUTO_ENCODER_1"
    config = json.load(open('confmod_auto_encoder.json'))[MODEL_NAME]
    # MODEL_NAME = "AUTO_ENCODER_1"
    # config = json.load(open('confmod_auto_encoder.json'))[MODEL_NAME]

    ''' Dataloading '''
    ### > Separate training and validation HDF5 files 
    ld = __import__("load_data")
    train_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "train", target_sample = 234)
    # validation_set = getattr(ld, config["DATA_LOADER"])(config, data_type = "valid")
    train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    # validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)

    ### > Single HDF5 file containing training and validation data 
    # data_set = load_data.Dataset_ae(config)
    # # train_set, validation_set, unused_set = torch.utils.data.random_split(data_set, [int(data_set.__len__()*0.3), int(data_set.__len__()*0.05), int(data_set.__len__()*0.65)], generator=torch.Generator().manual_seed(42))
    # train_set, validation_set = torch.utils.data.random_split(data_set, [int(data_set.__len__()*0.8), int(data_set.__len__()*0.2)], generator=torch.Generator().manual_seed(42))
    # train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    # validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=True)


    ''' Model setup '''
    wrapers = __import__("wrapers")
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)


    ''' Model loading from save file '''
    if config["continue"] == True:
        SAVEPATH = config["SAVEPATH"]
        checkpoint = torch.load(SAVEPATH)
        model.load_state_dict(checkpoint['state_dict'])
        print(" >>> Loaded checkpoint")


    ''' Logging and saving '''
    DATA_NAME = os.path.splitext(os.path.basename(config["PATH_TRAIN"]))[0]

    PATH = ""
    CONFIGURATION = f"../saves/{DATA_NAME}/save_{config['MODEL_NAME']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    # CONFIGURATION = f"../saves/save_{config['MODEL_NAME']}_Nodes{config['n_nodes']}_BS{config['batch_size']}_{datetime.datetime.now().date()}"
    logger = TensorBoardLogger(PATH, name=CONFIGURATION)


    # '''Define (pytorch_lightning) Trainer '''
    # ### > SLURM Training
    # trainer = pl.Trainer(max_epochs=config["epochs"], accelerator=config["device_type"], devices=config["devices"], num_nodes=config["num_nodes"], strategy='ddp', logger=logger)
    # ### > Jupyter Notebook Training
    # trainer = pl.Trainer(max_epochs=20, accelerator='gpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    # ### > Jupyter Notebook CPU Training
    trainer = pl.Trainer(max_epochs=500, accelerator='cpu', devices=1, strategy='auto', logger=logger, plugins=[LightningEnvironment()])
    
    ''' Train '''
    # trainer.fit(model, train_dataloader, validation_dataloader)
    trainer.fit(model, train_dataloader)

    ''' Saving configuration file into log folder ''' 
    LOGDIR = trainer.log_dir
    json_object = json.dumps(config, indent=4)
    with open(LOGDIR+"/config.json", "w") as outfile:
        outfile.write(json_object)




def main():
    train()

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/gpfs/data/fs72150/springerd/Projects/LuttingerWard_from_ML/data/U2c0_b10b50_gmax_10x10.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)